# ノートブック05: 決定木とアンサンブル - 複雑なパターンを学習する 🌳

**学習目標**: 決定木、Random Forest、Gradient Boostingの基礎を完全にマスターし、実データで適用できるようになる

---

## 📋 このノートブックで学ぶこと

### 1. 決定木の基礎 ⭐⭐⭐
- 決定木のアルゴリズム（CART）
- 情報利得とGini不純度の違い
- 木の深さと過学習の関係
- 決定木の可視化と解釈
- 剪定（Pruning）の概念

### 2. Random Forest（ランダムフォレスト） ⭐⭐⭐
- バギング（Bootstrap Aggregating）の仕組み
- ランダム特徴量選択
- Out-of-Bag（OOB）スコア
- 特徴量重要度の計算方法
- ハイパーパラメータの調整

### 3. Gradient Boosting基礎 ⭐⭐
- ブースティングの基本概念
- 残差学習の仕組み
- 学習率（learning_rate）の役割
- scikit-learnのGradientBoosting
- Random ForestとGradient Boostingの違い

### 4. 実践的な応用 ⭐⭐⭐
- 医療診断データでの分類
- モデルの比較と選択
- Cross-Validationによる評価
- 特徴量重要度の分析

---

## 🎯 達成目標

このノートブックを完了すると：
- ✅ 決定木の仕組みを完全に理解できる
- ✅ Random Forestを実装・最適化できる
- ✅ Gradient Boostingの基礎を理解できる
- ✅ 適切なアンサンブル手法を選択できる
- ✅ 実データで高精度なモデルを構築できる

**Note**: より高度なGBDT（LightGBM、XGBoost、CatBoost）はノートブック13-16で詳しく学びます

---

⏱️ **推定学習時間**: 120-150分  
📊 **難易度**: ★★★☆☆（中級）

---

## 🔧 1. セットアップ

In [ ]:
# 基本ライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path

# 機械学習
from sklearn.datasets import (
    make_classification, 
    make_moons, 
    load_breast_cancer,
    load_wine
)
from sklearn.model_selection import (
    train_test_split, 
    GridSearchCV, 
    cross_val_score,
    learning_curve
)
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix,
    roc_curve,
    roc_auc_score
)
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import (
    RandomForestClassifier, 
    GradientBoostingClassifier,
    BaggingClassifier
)

# 設定
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('Set2')
pd.set_option('display.max_columns', None)

# 再現性のためのシード
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✅ セットアップ完了")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## 🌲 2. 決定木の基礎

### 決定木とは？

決定木（Decision Tree）は、データを**if-thenルール**で分類・予測するモデルです。

**例**: タイタニック号の生存予測
```
性別は女性?
├─ Yes → 年齢 < 30?
│         ├─ Yes → 生存
│         └─ No  → 生存
└─ No  → チケットクラス = 1?
          ├─ Yes → 生存
          └─ No  → 死亡
```

### 決定木の利点と欠点

**利点** ✅:
- 解釈しやすい（ビジネスルールとして使える）
- 特徴量のスケーリング不要
- カテゴリカル変数を直接扱える
- 非線形な関係をモデル化できる
- 欠損値にロバスト

**欠点** ❌:
- 過学習しやすい
- 少しのデータ変化で木構造が大きく変わる（不安定）
- 線形な関係の表現が苦手

### 分割基準

#### 1. Gini不純度（Gini Impurity）
$$Gini = 1 - \sum_{i=1}^{C} p_i^2$$

- $p_i$: クラスiの割合
- 値が小さいほど「純粋」（同じクラスが集まっている）

#### 2. エントロピー（Entropy）
$$H = -\sum_{i=1}^{C} p_i \log_2(p_i)$$

- 情報理論に基づく指標
- 値が小さいほど「純粋」

### 2.1 シンプルな例：2次元分類

In [ ]:
# シンプルな2クラス分類データ生成
X, y = make_classification(
    n_samples=300,
    n_features=2,
    n_informative=2,
    n_redundant=0,
    n_clusters_per_class=1,
    class_sep=1.5,
    random_state=RANDOM_STATE
)

# データ分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=RANDOM_STATE
)

print("📊 データ情報:")
print(f"   訓練データ: {X_train.shape[0]} samples")
print(f"   テストデータ: {X_test.shape[0]} samples")
print(f"   特徴量数: {X.shape[1]}")
print(f"   クラス分布: {np.bincount(y)}")

# データの可視化
plt.figure(figsize=(10, 6))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='RdYlBu', edgecolors='black', alpha=0.7)
plt.xlabel('Feature 0')
plt.ylabel('Feature 1')
plt.title('分類データセット')
plt.colorbar(label='Class')
plt.grid(True, alpha=0.3)
plt.show()

### 2.2 決定木モデルの訓練

In [ ]:
# 決定木モデル（深さ制限あり）
dt = DecisionTreeClassifier(
    max_depth=3,
    criterion='gini',  # 'gini' or 'entropy'
    random_state=RANDOM_STATE
)
dt.fit(X_train, y_train)

# 予測
y_train_pred = dt.predict(X_train)
y_test_pred = dt.predict(X_test)

# 評価
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)

print("📊 決定木の結果:")
print(f"\n訓練精度: {train_acc:.3f}")
print(f"テスト精度: {test_acc:.3f}")
print(f"\n木の深さ: {dt.get_depth()}")
print(f"葉の数: {dt.get_n_leaves()}")
print(f"\n詳細レポート:")
print(classification_report(y_test, y_test_pred, target_names=['Class 0', 'Class 1']))

### 2.3 決定木の可視化 ⭐⭐⭐

In [ ]:
# 木構造の可視化
plt.figure(figsize=(20, 10))
plot_tree(
    dt, 
    feature_names=['Feature 0', 'Feature 1'],
    class_names=['Class 0', 'Class 1'],
    filled=True,
    rounded=True,
    fontsize=12
)
plt.title('決定木の構造（max_depth=3）', fontsize=16)
plt.show()

print("\n💡 木の読み方:")
print("   - 各ノード: 分割条件")
print("   - gini: Gini不純度")
print("   - samples: サンプル数")
print("   - value: クラスごとのサンプル数")
print("   - class: 多数決クラス")

### 2.4 決定境界の可視化

In [ ]:
def plot_decision_boundary(model, X, y, title='Decision Boundary'):
    """
    決定境界を可視化する関数
    """
    h = 0.02  # メッシュのステップサイズ
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    
    # 各点での予測
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # プロット
    plt.figure(figsize=(10, 6))
    plt.contourf(xx, yy, Z, alpha=0.4, cmap='RdYlBu')
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap='RdYlBu', edgecolors='black', s=50)
    plt.xlabel('Feature 0')
    plt.ylabel('Feature 1')
    plt.title(title)
    plt.colorbar(label='Class')
    plt.grid(True, alpha=0.3)
    plt.show()

# 決定境界を表示
plot_decision_boundary(dt, X_test, y_test, '決定木の決定境界（max_depth=3）')

print("\n💡 ポイント: 決定木の境界は直線（または平面）の組み合わせ")

### 2.5 深さと過学習の関係 ⭐⭐⭐

In [ ]:
# 異なる深さで訓練し、精度の変化を観察
max_depths = range(1, 20)
train_scores = []
test_scores = []

for depth in max_depths:
    dt_temp = DecisionTreeClassifier(max_depth=depth, random_state=RANDOM_STATE)
    dt_temp.fit(X_train, y_train)
    train_scores.append(dt_temp.score(X_train, y_train))
    test_scores.append(dt_temp.score(X_test, y_test))

# 可視化
plt.figure(figsize=(12, 6))
plt.plot(max_depths, train_scores, 'o-', label='訓練スコア', linewidth=2, markersize=6)
plt.plot(max_depths, test_scores, 's-', label='テストスコア', linewidth=2, markersize=6)
plt.xlabel('木の最大深さ（max_depth）', fontsize=12)
plt.ylabel('精度（Accuracy）', fontsize=12)
plt.title('木の深さと精度の関係', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.axvline(x=max_depths[np.argmax(test_scores)], color='red', linestyle='--', alpha=0.5, label='最適な深さ')
plt.show()

best_depth = max_depths[np.argmax(test_scores)]
print(f"\n📊 結果:")
print(f"   最適な深さ: {best_depth}")
print(f"   最高テスト精度: {max(test_scores):.3f}")
print(f"\n💡 重要なポイント:")
print(f"   - 深さ < {best_depth}: 過小学習（underfitting）")
print(f"   - 深さ = {best_depth}: 最適")
print(f"   - 深さ > {best_depth}: 過学習（overfitting）")

## 🌲🌲🌲 3. Random Forest（ランダムフォレスト）

### Random Forestとは？

Random Forestは、**複数の決定木を組み合わせる**アンサンブル手法です。

**3つのランダム性**:
1. **バギング（Bootstrap Aggregating）**: 訓練データをランダムサンプリング
2. **ランダム特徴量選択**: 各分岐で一部の特徴量のみ使用
3. **多数決**: 複数の木の予測を平均化

**アルゴリズム**:
```
1. N本の決定木を準備
2. 各木に対して:
   a. ブートストラップサンプリングで訓練データを作成
   b. 各分岐で√m個の特徴量をランダム選択（mは全特徴量数）
   c. 決定木を訓練
3. 予測時: N本の木の多数決（分類）または平均（回帰）
```

### Random Forestの利点

- ✅ 過学習しにくい（単一の決定木より）
- ✅ 高精度
- ✅ 並列処理可能（各木は独立）
- ✅ 特徴量重要度が得られる
- ✅ Out-of-Bag（OOB）スコアで検証不要
- ✅ 欠損値にロバスト

### 3.1 Random Forestの訓練

In [ ]:
# Random Forestモデル
rf = RandomForestClassifier(
    n_estimators=100,      # 木の数
    max_depth=5,           # 各木の最大深さ
    min_samples_split=2,   # 分割に必要な最小サンプル数
    min_samples_leaf=1,    # 葉に必要な最小サンプル数
    max_features='sqrt',   # 各分岐で使う特徴量数（√m）
    bootstrap=True,        # ブートストラップサンプリング
    oob_score=True,        # OOBスコアを計算
    random_state=RANDOM_STATE,
    n_jobs=-1              # 並列処理（全CPUを使用）
)
rf.fit(X_train, y_train)

# 予測
y_train_pred_rf = rf.predict(X_train)
y_test_pred_rf = rf.predict(X_test)

# 評価
train_acc_rf = accuracy_score(y_train, y_train_pred_rf)
test_acc_rf = accuracy_score(y_test, y_test_pred_rf)
oob_score = rf.oob_score_

print("📊 Random Forestの結果:")
print(f"\n訓練精度: {train_acc_rf:.3f}")
print(f"テスト精度: {test_acc_rf:.3f}")
print(f"OOBスコア: {oob_score:.3f}")
print(f"\n木の数: {rf.n_estimators}")
print(f"各木の平均深さ: {np.mean([tree.get_depth() for tree in rf.estimators_]):.1f}")

print("\n💡 OOB（Out-of-Bag）スコアとは:")
print("   各木の訓練で使われなかったサンプルでの精度")
print("   検証データ不要でモデルの汎化性能を推定できる")

### 3.2 決定境界の比較

In [ ]:
# Random Forestの決定境界
plot_decision_boundary(rf, X_test, y_test, 'Random Forestの決定境界（100本の木）')

print("\n💡 ポイント: Random Forestの境界は単一の決定木より滑らか")

### 3.3 木の数（n_estimators）の影響

In [ ]:
# 木の数を変えて実験
n_estimators_list = [1, 5, 10, 20, 50, 100, 200, 500]
train_scores_rf = []
test_scores_rf = []
oob_scores_rf = []

print("🔄 異なる木の数で実験中...")
for n_est in n_estimators_list:
    rf_temp = RandomForestClassifier(
        n_estimators=n_est,
        max_depth=5,
        oob_score=True,
        random_state=RANDOM_STATE,
        n_jobs=-1
    )
    rf_temp.fit(X_train, y_train)
    train_scores_rf.append(rf_temp.score(X_train, y_train))
    test_scores_rf.append(rf_temp.score(X_test, y_test))
    oob_scores_rf.append(rf_temp.oob_score_)

# 可視化
plt.figure(figsize=(12, 6))
plt.plot(n_estimators_list, train_scores_rf, 'o-', label='訓練スコア', linewidth=2)
plt.plot(n_estimators_list, test_scores_rf, 's-', label='テストスコア', linewidth=2)
plt.plot(n_estimators_list, oob_scores_rf, '^-', label='OOBスコア', linewidth=2)
plt.xscale('log')
plt.xlabel('木の数（n_estimators）', fontsize=12)
plt.ylabel('精度（Accuracy）', fontsize=12)
plt.title('Random Forest: 木の数と精度', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n💡 重要なポイント:")
print(f"   - 木の数が増えると精度が安定する")
print(f"   - 過学習しにくい（訓練とテストの差が小さい）")
print(f"   - 100本程度で十分な場合が多い")

## ⚡ 4. Gradient Boosting基礎

### Gradient Boostingとは？

Gradient Boostingは、**弱い学習器を順次追加**していく手法です。

**Random Forestとの違い**:
- **Random Forest**: 木を**並列**に訓練（バギング）
- **Gradient Boosting**: 木を**逐次**に訓練（ブースティング）

**アルゴリズム**:
```
1. 初期予測 f₀(x) を設定（例: 平均値）
2. for m = 1 to M:
   a. 残差 rᵢ = yᵢ - f_{m-1}(xᵢ) を計算
   b. 残差を予測する木 hₘ を訓練
   c. f_m(x) = f_{m-1}(x) + η · hₘ(x)  (ηは学習率)
3. 最終予測: f_M(x)
```

**重要なパラメータ**:
- `n_estimators`: 木の数（多いほど良いが、計算時間増加）
- `learning_rate`: 学習率（小さいほど慎重、多くの木が必要）
- `max_depth`: 各木の深さ（通常3-5程度）

**Note**: より高度なGBDT実装（LightGBM、XGBoost、CatBoost）はノートブック13-16で詳しく学びます

### 4.1 Gradient Boostingの訓練

In [ ]:
# Gradient Boostingモデル
gb = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    min_samples_split=2,
    min_samples_leaf=1,
    subsample=1.0,  # サブサンプリング率（1.0 = 全データ使用）
    random_state=RANDOM_STATE
)
gb.fit(X_train, y_train)

# 予測
y_train_pred_gb = gb.predict(X_train)
y_test_pred_gb = gb.predict(X_test)

# 評価
train_acc_gb = accuracy_score(y_train, y_train_pred_gb)
test_acc_gb = accuracy_score(y_test, y_test_pred_gb)

print("📊 Gradient Boostingの結果:")
print(f"\n訓練精度: {train_acc_gb:.3f}")
print(f"テスト精度: {test_acc_gb:.3f}")
print(f"\n木の数: {gb.n_estimators}")
print(f"学習率: {gb.learning_rate}")

# 決定境界
plot_decision_boundary(gb, X_test, y_test, 'Gradient Boostingの決定境界')

### 4.2 学習率（learning_rate）の影響

In [ ]:
# 異なる学習率で実験
learning_rates = [0.01, 0.05, 0.1, 0.5, 1.0]
train_scores_gb = []
test_scores_gb = []

print("🔄 異なる学習率で実験中...")
for lr in learning_rates:
    gb_temp = GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=lr,
        max_depth=3,
        random_state=RANDOM_STATE
    )
    gb_temp.fit(X_train, y_train)
    train_scores_gb.append(gb_temp.score(X_train, y_train))
    test_scores_gb.append(gb_temp.score(X_test, y_test))

# 可視化
plt.figure(figsize=(12, 6))
plt.plot(learning_rates, train_scores_gb, 'o-', label='訓練スコア', linewidth=2)
plt.plot(learning_rates, test_scores_gb, 's-', label='テストスコア', linewidth=2)
plt.xscale('log')
plt.xlabel('学習率（learning_rate）', fontsize=12)
plt.ylabel('精度（Accuracy）', fontsize=12)
plt.title('Gradient Boosting: 学習率と精度', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print("\n💡 学習率のトレードオフ:")
print("   - 小さい（例: 0.01）: 慎重に学習、多くの木が必要、過学習しにくい")
print("   - 大きい（例: 1.0）: 素早く学習、少ない木でOK、過学習しやすい")
print("   - 推奨: 0.05-0.1程度")

## 📊 5. モデル比較

決定木、Random Forest、Gradient Boostingを総合的に比較します。

In [ ]:
# 3つのモデルを同じデータで訓練・評価
models = {
    'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=RANDOM_STATE),
    'Random Forest': RandomForestClassifier(n_estimators=100, max_depth=5, random_state=RANDOM_STATE, n_jobs=-1),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=RANDOM_STATE)
}

results = []
for name, model in models.items():
    # 訓練
    model.fit(X_train, y_train)
    
    # 評価
    train_acc = model.score(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    
    # Cross-Validation
    cv_scores = cross_val_score(model, X, y, cv=5)
    
    results.append({
        'Model': name,
        'Train Acc': train_acc,
        'Test Acc': test_acc,
        'CV Acc (mean)': cv_scores.mean(),
        'CV Acc (std)': cv_scores.std(),
        'Overfit': train_acc - test_acc
    })

# 結果をDataFrameに
results_df = pd.DataFrame(results)
print("\n📊 モデル比較結果:")
display(results_df)

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 精度比較
x = np.arange(len(results_df))
width = 0.35
axes[0].bar(x - width/2, results_df['Train Acc'], width, label='Train', alpha=0.8)
axes[0].bar(x + width/2, results_df['Test Acc'], width, label='Test', alpha=0.8)
axes[0].set_ylabel('Accuracy')
axes[0].set_title('訓練精度 vs テスト精度')
axes[0].set_xticks(x)
axes[0].set_xticklabels(results_df['Model'])
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# 過学習度
axes[1].bar(results_df['Model'], results_df['Overfit'], color=['red', 'orange', 'green'], alpha=0.7)
axes[1].set_ylabel('Overfitting (Train - Test)')
axes[1].set_title('過学習の度合い（低いほど良い）')
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

print("\n💡 まとめ:")
best_model = results_df.loc[results_df['Test Acc'].idxmax(), 'Model']
print(f"   最高テスト精度: {best_model}")
print(f"   最小過学習: {results_df.loc[results_df['Overfit'].idxmin(), 'Model']}")

## 🏥 6. 実践例: Breast Cancer診断

実データでモデルを適用し、医療診断の精度を向上させます。

In [ ]:
# Breast Cancerデータセット読み込み
data = load_breast_cancer()
X_cancer = data.data
y_cancer = data.target
feature_names_cancer = data.feature_names

print("🏥 Breast Cancer Dataset")
print(f"\nサンプル数: {X_cancer.shape[0]}")
print(f"特徴量数: {X_cancer.shape[1]}")
print(f"\nクラス分布:")
class_counts = pd.Series(y_cancer).value_counts()
print(f"   Malignant (悪性): {class_counts[0]} ({class_counts[0]/len(y_cancer)*100:.1f}%)")
print(f"   Benign (良性): {class_counts[1]} ({class_counts[1]/len(y_cancer)*100:.1f}%)")

# データ分割
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_cancer, y_cancer, test_size=0.3, random_state=RANDOM_STATE, stratify=y_cancer
)

print(f"\n訓練データ: {X_train_c.shape[0]} samples")
print(f"テストデータ: {X_test_c.shape[0]} samples")

### 6.1 Random Forestで診断

In [ ]:
# Random Forestモデル
rf_cancer = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=RANDOM_STATE,
    n_jobs=-1
)
rf_cancer.fit(X_train_c, y_train_c)

# 予測
y_pred_c = rf_cancer.predict(X_test_c)
y_pred_proba_c = rf_cancer.predict_proba(X_test_c)[:, 1]

# 評価
print("📊 Random Forest診断結果:")
print(f"\n精度: {accuracy_score(y_test_c, y_pred_c):.3f}")
print(f"AUC-ROC: {roc_auc_score(y_test_c, y_pred_proba_c):.3f}")
print(f"\n詳細レポート:")
print(classification_report(y_test_c, y_pred_c, target_names=['Malignant', 'Benign']))

# Confusion Matrix
cm = confusion_matrix(y_test_c, y_pred_c)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Malignant', 'Benign'],
            yticklabels=['Malignant', 'Benign'],
            cbar_kws={'label': 'Count'})
plt.ylabel('Actual', fontsize=12)
plt.xlabel('Predicted', fontsize=12)
plt.title('Confusion Matrix - Breast Cancer Diagnosis', fontsize=14)
plt.show()

print("\n💡 医療診断での重要ポイント:")
print("   - False Negative（悪性を良性と誤診）: 最も避けたい")
print("   - False Positive（良性を悪性と誤診）: 不要な検査が増える")
print(f"   - 現在のFalse Negative: {cm[0, 1]} 件")
print(f"   - 現在のFalse Positive: {cm[1, 0]} 件")

### 6.2 特徴量重要度の分析 ⭐⭐⭐

In [ ]:
# 特徴量重要度の取得
feature_importance = pd.DataFrame({
    'Feature': feature_names_cancer,
    'Importance': rf_cancer.feature_importances_
}).sort_values('Importance', ascending=False)

# Top 15特徴量を可視化
top_features = feature_importance.head(15)

plt.figure(figsize=(12, 8))
plt.barh(range(len(top_features)), top_features['Importance'], color='steelblue', alpha=0.8)
plt.yticks(range(len(top_features)), top_features['Feature'])
plt.xlabel('重要度（Importance）', fontsize=12)
plt.title('Top 15 重要な特徴量 - Breast Cancer診断', fontsize=14)
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\n📊 Top 10 重要な特徴量:")
display(top_features.head(10))

print("\n💡 特徴量重要度の活用:")
print("   - 医師への説明: どの指標が診断に重要か")
print("   - 特徴量選択: 重要度の低い特徴量を削除")
print("   - モデルの解釈: ビジネスロジックの理解")

### 6.3 ROC曲線

In [ ]:
# ROC曲線のプロット
fpr, tpr, thresholds = roc_curve(y_test_c, y_pred_proba_c)
roc_auc = roc_auc_score(y_test_c, y_pred_proba_c)

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve - Breast Cancer Diagnosis', fontsize=14)
plt.legend(loc="lower right", fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print("\n💡 ROC曲線の読み方:")
print("   - AUC（曲線下面積）が1に近いほど良いモデル")
print(f"   - 現在のAUC: {roc_auc:.3f} → {'優秀' if roc_auc > 0.9 else '良好'}")
print("   - ランダム分類器（点線）より大幅に上")

### 6.4 ハイパーパラメータチューニング

In [ ]:
# GridSearchCVでパラメータ探索
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2']
}

grid_search = GridSearchCV(
    RandomForestClassifier(random_state=RANDOM_STATE),
    param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=0
)

print("\n🔍 GridSearchCV実行中...")
print(f"   探索するパラメータ組み合わせ: {len(param_grid['n_estimators']) * len(param_grid['max_depth']) * len(param_grid['min_samples_split']) * len(param_grid['min_samples_leaf']) * len(param_grid['max_features'])}")

grid_search.fit(X_train_c, y_train_c)

print(f"\n✅ 最適なパラメータ:")
for param, value in grid_search.best_params_.items():
    print(f"   {param}: {value}")

print(f"\n最高CVスコア: {grid_search.best_score_:.4f}")
print(f"テストスコア: {grid_search.score(X_test_c, y_test_c):.4f}")

# 最良モデルの詳細評価
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test_c)

print(f"\n最適化後の詳細レポート:")
print(classification_report(y_test_c, y_pred_best, target_names=['Malignant', 'Benign']))

## 🎓 7. まとめ

### 学んだこと

1. ✅ **決定木**
   - if-thenルールで解釈しやすい
   - 過学習しやすいため深さ制御が重要
   - 非線形パターンをモデル化可能

2. ✅ **Random Forest**
   - バギング + ランダム特徴量選択
   - 過学習しにくく、高精度
   - 並列処理可能で高速
   - 特徴量重要度が得られる

3. ✅ **Gradient Boosting**
   - 逐次的に木を追加
   - 高精度だが調整が必要
   - 学習率とのトレードオフ

4. ✅ **実践的なテクニック**
   - GridSearchCVでパラメータ最適化
   - Cross-Validationで汎化性能評価
   - 特徴量重要度で解釈性向上

### モデルの使い分け

| モデル | 利点 | 欠点 | 推奨用途 |
|--------|------|------|----------|
| **決定木** | 解釈しやすい<br>高速 | 過学習しやすい<br>不安定 | 探索的分析<br>ベースライン |
| **Random Forest** | 安定<br>高精度<br>並列化可能 | 解釈性やや低い<br>メモリ消費大 | 一般的な分類・回帰<br>特徴量選択 |
| **Gradient Boosting** | 最高精度<br>柔軟性高い | 調整が複雑<br>過学習リスク | コンペティション<br>精度重視タスク |

### 次のステップ

- **ノートブック06**: SVM とカーネル法
- **ノートブック13-16**: 高度なGBDT（LightGBM、XGBoost、CatBoost）
  - より高速なアルゴリズム
  - カテゴリカル変数の直接処理
  - GPU対応
  - Kaggleで頻繁に使用される

### 重要なポイント

💡 **いつ使うべきか？**
- テーブルデータ（表形式データ）
- 特徴量が多い場合
- 非線形な関係がある場合
- 解釈性と精度の両方が必要な場合

⚠️ **注意点**
- 画像・音声・テキストには不向き（ディープラーニング推奨）
- データが少ない場合は線形モデルも検討
- 計算リソースを考慮（Random Forestは並列化可能）

---

**🎉 お疲れ様でした！決定木とアンサンブルの基礎から実践までマスターしました！**

## 📝 練習問題

### 問題1: 基礎知識
1. Random ForestとGradient Boostingの違いを説明してください
2. なぜRandom Forestは過学習しにくいのですか？
3. Gini不純度とエントロピーの違いは何ですか？

### 問題2: 実装
1. Wineデータセット（`load_wine()`）でRandom Forestを訓練してください
2. GridSearchCVを使って最適なパラメータを見つけてください
3. 特徴量重要度を可視化し、上位5つの特徴量を特定してください

### 問題3: 発展
1. `max_features`パラメータを変えて、モデルの精度がどう変わるか実験してください
2. Random ForestとGradient Boostingで同じデータセットを比較し、どちらが良いか考察してください
3. 学習曲線（Learning Curve）をプロットして、モデルの改善余地を分析してください

---

## 🔗 参考リンク

- [scikit-learn Decision Trees](https://scikit-learn.org/stable/modules/tree.html)
- [scikit-learn Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
- [scikit-learn Gradient Boosting](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)
- [Understanding Random Forests](https://www.stat.berkeley.edu/~breiman/randomforest2001.pdf)

---

**Happy Learning! 🌳**